### Clase mensaje

In [2]:
#Clase donde vamos a ir metiendo todos los fragmentos de cada mensaje
class Mensaje: 
    idmensaje = 0
    frame = ""
    principal = ""
    nbytes = ""
    ncolumnas = 0
    nfilas = 0
    matriz = []
    end = ""
    
    macdest = "" #Direccion MAC Destino
    macorig = "" #Dirección MAC Origen
    mactipo = "" #Tipo de mensaje que utiliza
    
    arphwtype = "" #Tipo de hardware usado
    arpprotocoltype = "" #Tipo de protocolo ARP
    arphwsize = "" #Tamaño del hardware ARP
    arpprotocolsize = "" #Tamaño del protocolo ARP
    arpopcoderequest = "" #Código request ARP
    arpsourcemac = "" #MAC origen
    arpsourceip = "" #IP origen
    arptargetmac = "" #MAC objetivo
    arptargetip = "" #IP objetivo
    arppadding = "" #Padding ARP
    esarp=False
    
    ipversion = "" #Version de ip utilizada
    ipsize = "" #Tamaño data grama IP
    ipdifservfield = "" #Differenciated service field
    iptotallen = "" #Tamaño total desde mensaje IP
    ipid = "" #Identificacion
    ipflags = "" #Flags
    ipttl = "" #Time to life del ip
    ipprotocolo = "" #Tipo de protocolo que utiliza el mensaje IP
    ipheaderchecksum = "" #Checksum de la cabecera ip
    ipdest = "" #Dirección IP Destino
    iporig = "" #Dirección IP Origen
    ipv6trafficclass = "" #Clase de tráfico IPv6
    ipv6nextheader = "" #Próxima cabecera IPv6
    ipv6hoplimit = "" #Hop limit IPv6
    esip = False
    
    tcpsourceport = "" #Puerto origen para el datagrama TCP
    tcpdestport = "" #Puerto destino para el datagrama TCP
    tcpsequencenumber = "" #Número de secuencia del datagrama TCP
    tcpacknowledgenumber = "" #Número de acknowledge del datagrama TCP
    tcpheaderlenght = "" #Longitud de cabecera del datagrama TCP
    tcpflag = "" #Flag del datagrama TCP
    tcpwinsizevalue = "" #Tamaño de ventana del datagrama TCP
    tcpchecksum = "" #Checksum del datagrama TCP
    tcpurgentpointer = "" #Urgent pinter del datagrama TCP
    tcpmaxsegmentsize = "" #Tamaño máximo del segmento del datagrama TCP
    tcpsackoption = "" #Opcion sack del datagrama TCP
    tcptimestamps = "" #Timestamos del datagrama TCP
    tcpnop = "" #Operacion no del datagrama TCP
    tcpwindowscale = "" #Escala de ventana del datagrama TCP
    estcp = False
    
    icmptype = "" #Tipo de mensaje ICMP
    icmpcode = "" #Código del mesaje ICMP
    icmpchecksum = "" #Checksum del ICMP
    icmpid = "" #Id ICMP
    icmpseq = "" #Secuencia ICMP
    esicmp=False
    
    udpsourceport = "" #Puerto origen UDP  
    udpdestport = "" #Puerto destino UDP
    udplength = "" #Longitud mensaje UDP
    udpchecksum = "" #Checksum UDP
    esudp = False
    
    dnstransid = "" #ID tansacción DNS
    dnsflags = "" #Flags DNS
    dnsquestions = "" #Preguntas DNS
    dnsanswer = "" #Respuestas DNS
    dnsauthority = "" #Autoridad DNS
    dnsadditional = "" #Adicional DNS
    dnsqueryname = "" #nombre consulta DNS
    dnsquerytype = "" #tipo consulta DNS
    dnsqueryclass = "" #clase consulta DNS
    esdns = False

### Iniciador

In [3]:
#Esta función es necesaria para que los mensajes no se solapen cuando se asignan las partes de cada mensaje. 
#Si no se inicializa, las siguientes variables contendrán los elementos de todos los mensajes.
#En otras palabras, MAC destino será una lista con todas las MAC destino halladas. Todos los apartados de cada mensaje serán iguales
def iniciador(m):
    m.idmensaje = 0
    m.frame = ""
    m.principal = ""
    m.nbytes = ""
    m.ncolumnas = 0
    m.nfilas = 0
    m.matriz = []
    m.end = ""
    
    m.macdest = "" #Direccion MAC Destino
    m.macorig = "" #Dirección MAC Origen
    m.mactipo = "" #Tipo de mensaje que utiliza
    
    m.arphwtype = "" #Tipo de hardware usado
    m.arpprotocoltype = "" #Tipo de protocolo ARP
    m.arphwsize = "" #Tamaño del hardware ARP
    m.arpprotocolsize = "" #Tamaño del protocolo ARP
    m.arpopcoderequest = "" #Código request ARP
    m.arpsourcemac = "" #MAC origen
    m.arpsourceip = "" #IP origen
    m.arptargetmac = "" #MAC objetivo
    m.arptargetip = "" #IP objetivo
    m.arppadding = "" #Padding ARP
    m.esarp=False
    
    m.ipversion = "" #Version de ip utilizada
    m.ipsize = "" #Tamaño data grama IP
    m.ipdifservfield = "" #Differenciated service field
    m.iptotallen = "" #Tamaño total desde mensaje IP
    m.ipid = "" #Identificacion
    m.ipflags = "" #Flags
    m.ipttl = "" #Time to life del ip
    m.ipprotocolo = "" #Tipo de protocolo que utiliza el mensaje IP
    m.ipheaderchecksum = "" #Checksum de la cabecera ip
    m.ipdest = "" #Dirección IP Destino
    m.iporig = "" #Dirección IP Origen
    m.ipv6trafficclass = "" #Clase de tráfico IPv6
    m.ipv6nextheader = "" #Próxima cabecera IPv6
    m.ipv6hoplimit = "" #Hop limit IPv6
    m.esip = False
        
    m.tcpsourceport = "" #Puerto origen para el datagrama TCP
    m.tcpdestport = "" #Puerto destino para el datagrama TCP
    m.tcpsequencenumber = "" #Número de secuencia del datagrama TCP
    m.tcpacknowledgenumber = "" #Número de acknowledge del datagrama TCP
    m.tcpheaderlenght = "" #Longitud de cabecera del datagrama TCP
    m.tcpflag = "" #Flag del datagrama TCP
    m.tcpwinsizevalue = "" #Tamaño de ventana del datagrama TCP
    m.tcpchecksum = "" #Checksum del datagrama TCP
    m.tcpurgentpointer = "" #Urgent pinter del datagrama TCP
    m.tcpmaxsegmentsize = "" #Tamaño máximo del segmento del datagrama TCP
    m.tcpsackoption = "" #Opcion sack del datagrama TCP
    m.tcptimestamps = "" #Timestamos del datagrama TCP
    m.tcpnop = "" #Operacion no del datagrama TCP
    m.tcpwindowscale = "" #Escala de ventana del datagrama TCP
    m.estcp = False
    
    m.icmptype = "" #Tipo de mensaje ICMP
    m.icmpcode = "" #Código del mesaje ICMP
    m.icmpchecksum = "" #Checksum del ICMP
    m.icmpid = "" #Id ICMP
    m.icmpseq = "" #Secuencia ICMP
    m.esicmp=False
    
    m.udpsourceport = "" #Puerto origen UDP  
    m.udpdestport = "" #Puerto destino UDP
    m.udplength = "" #Longitud mensaje UDP
    m.udpchecksum = "" #Checksum UDP
    m.esudp = False

    m.dnstransid = "" #ID tansacción DNS
    m.dnsflags = "" #Flags DNS
    m.dnsquestions = "" #Preguntas DNS
    m.dnsanswer = "" #Respuestas DNS
    m.dnsauthority = "" #Autoridad DNS
    m.dnsadditional = "" #Adicional DNS
    m.dnsqueryname = "" #nombre consulta DNS
    m.dnsquerytype = "" #tipo consulta DNS
    m.dnsqueryclass = "" #clase consulta DNS
    m.esdns = False

#### Parseo de los elementos del archivo carrays

In [98]:
f = open("arpping", "r") #Abrimos el documento generado
mensajesarchivo = [] #Lista donde guardaremos todos los mensajes del archivo
acabado = False #Booleano para idicar cuando hemos acabado de leer el archivo
contador = 1 #Contador que nos indica el número de mensaje que estamos leyendo

while (acabado == False):
    msg = Mensaje() #asignamos a msg la estructura del mensaje
    iniciador(msg) #Constructora

    msg.frame = f.readline() #Leemos la primera linea
    if (msg.frame == ""): #Si está vacío quiere decir que ya no hay más mensajes
        acabado = True
    else:    
        msg.principal = f.readline() #Leemos la descripción del paquete
        aux = msg.principal.split() #Separamos y tratamos la línea
        aux = aux[4].split('[') #Separamos el numero del mensaje del número de bytes
        msg.idmensaje = contador #Id. del mensaje que será igual al contador que llevemos
        
        msg.nbytes = aux[1].split(']') #Obtenemos el número de bytes de la primera línea
        msg.nbytes = int(msg.nbytes[0]) #Convertimos el número a entero
        msg.ncolumnas = 8 #El número de columnas siempre es 8
        msg.nfilas = int(msg.nbytes/msg.ncolumnas) #Dividimos entre el numero de columnas para obtener el número de filas
        if (msg.nbytes%8 != 0): #Si el numero de bytes no es multiplo de 8, le añadimos una fila más
            msg.nfilas += 1

        i=0
        #Bucle para obtener todos los datos del datagrama
        for linea in f:
            j=0 #Inicializamos j al comenzar la fila
            for caracter in linea.split(', '):
                if (j < 8):
                    msg.matriz.append(caracter[2:4]) #Para que coja solamente los 4 caracteres que queremos (se usa para el último caso sobre todo)
                    j+=1 #Incrementamos numero de columnas

            i+=1 #Incrementamos numero de filas
            if(i==msg.nfilas): #Si el numero de filas es igual al limite, paramos la lectura
                break
        msg.end = f.readline() #Cogemos el }; final
        f.readline()#Salto de línea
        mensajesarchivo.append(msg) #Añadimos el mensaje a la lista de mensajes
        contador += 1
f.close()

### Elementos del mensaje

In [99]:
def elementos(m):
    print("ID MENSAJE: ",m.idmensaje)
    print("Es IP: ",m.esip)
    print("Es ARP: ",m.esarp)
    print("Es ICMP: ",m.esicmp)
    print("Es TCP: ",m.estcp)
    print("Es UDP: ",m.esudp)
    print("Es DNS: ",m.esdns)
    #Atributos MAC
    print ("MAC Destino: ",m.macdest)
    print ("MAC Origen: ",m.macorig)
    print ("Tipo MAC: ",m.mactipo)
    
    if(m.esip): 
        #Atributos IP
        print ("VersionIP: ",m.ipversion)
        if (m.ipversion == "4"):
            print ("Tamaño IP: ",m.ipsize)
            print ("Differenciated Service Field IP: ",m.ipdifservfield)
            print ("Tamaño total mensaje IP: ",m.iptotallen)
            print ("Identificacion IP: ",m.ipid)
            print ("Flags IP: ",m.ipflags)
            print ("Time to life IP: ",m.ipttl)
            print ("Protocolo IP: ",m.ipprotocolo)
            print ("Checksum cabecera IP: ",m.ipheaderchecksum)
            print ("IP Destino: ",m.ipdest)
            print ("IP Origen: ",m.iporig)
        elif (m.ipversion == "6"):
            print ("Clase de tráfico IPv6: ",m.ipv6trafficclass)
            print ("Longitud total de mensaje IPv6: ",m.iptotallen)
            print ("Próxima cabecera IPv6: ",m.ipv6nextheader)
            print ("Hoplimit IPV6: ",m.ipv6hoplimit)
            print ("IPV6 Destino: ",m.ipdest)
            print ("IPv6 Origen: ",m.iporig)
            
    if (m.esarp):
        print("Tipo hardware ARP: ",m.arphwtype)
        print("Protocolo ARP: ",m.arpprotocoltype)
        print("Tamaño Hardware ARP: ",m.arphwsize)
        print("Ramaño protocolo ARP: ",m.arpprotocolsize)
        print("Code request ARP: ",m.arpopcoderequest)
        print("MAC origen ARP: ",m.arpsourcemac)
        print("IP origen ARP: ",m.arpsourceip)
        print("MAC objetivo ARP: ",m.arptargetmac)
        print("IP objetivo ARP: ",m.arptargetip)
        print("Padding ARP: ",m.arppadding)
    if(m.esicmp):
        #Atributos ICMP
        print ("Tipo ICMP: ",m.icmptype)
        print ("Código ICMP: ",m.icmpcode)
        print ("Checksum ICMP: ",m.icmpchecksum)
        print ("Id. ICMP: ",m.icmpid)
        print ("secuencia ICMP: ",m.icmpseq)
    
    if(m.estcp):
        #Atributos TCP
        print("Puerto origen TCP: ",m.tcpsourceport)
        print("Puerto destino TCP: ",m.tcpdestport)
        print("Número de secuencia TCP: ",m.tcpsequencenumber)
        print("Numero de ACK TCP: ",m.tcpacknowledgenumber)
        print("Longitud de cabecera TCP: ",m.tcpheaderlenght)
        print("Flag TCP: ",m.tcpflag)
        print("Tamaño de ventana TCP: ",m.tcpwinsizevalue)
        print("Checksum TCP: ",m.tcpchecksum)
        print("Urgent pointer TCP: ",m.tcpurgentpointer)
        print("Tamaño máximo de segmento TCP: ",m.tcpmaxsegmentsize)
        print("Opción Sack TCP: ",m.tcpsackoption)
        print("Timestamps TCP: ",m.tcptimestamps)
        print("NoOperation TCP: ",m.tcpnop)
        print("Escala de ventana TCP: ",m.tcpwindowscale)
    
    if(m.esudp):
        #Atributos UDP
        print("Puerto origen UDP: ",m.udpsourceport)
        print("Puerto destino UDP: ",m.udpdestport)
        print("Longitud UDP: ",m.udplength)
        print("Checksum UDP: ",m.udpchecksum)
    
    if(m.esdns):
        #Atributos DNS
        print("Id. de transacción DNS DNS: ",m.dnstransid)
        print("Flags DNS: ",m.dnsflags)
        print("Preguntas DNS: ",m.dnsquestions)
        print("Respuestas DNS: ", m.dnsanswer)
        print("Autoridad DNS: ",m.dnsauthority) 
        print("Adicional DNS: ",m.dnsadditional)
        print("Nombre query DNS",m.dnsqueryname) 
        print("Tipo query DNS",m.dnsquerytype)
        print("Clase query DNS",m.dnsqueryclass)

    print("------------------------------------------------- \n")

### Conversor Hexadecimal a decimal

In [100]:
def conversor(hexa):
    long = len(hexa) - 1 #Guardamos la longitud del número que nos pasan
    decimal = 0 #Variable para guardar el número decimal 
    for i in range(len(hexa)): #Vamos analizando cada número decimal
        #Si son letras, las pasamos según su valor y posición a sistema decimal
        if (hexa[long-i] == "a"):
            decimal += 10 * (16**i)
        elif (hexa[long-i] == "b"):
            decimal += 11 * (16**i)
        elif (hexa[long-i] == "c"):
            decimal += 12 * (16**i)
        elif (hexa[long-i] == "d"):
            decimal += 13 * (16**i)
        elif (hexa[long-i] == "e"):
            decimal += 14 * (16**i)
        elif (hexa[long-i] == "f"):
            decimal += 15 * (16**i)
        else:
            decimal += int(hexa[long-i]) * (16**i)
             
    return decimal

### Conversor hexadecimal a dirección IP

In [101]:
def conversorip(direccion):
    resultado = [] #Lista para guardar la dirección
    nueva = "" #Resultado que devolveremos
    for fragmento in direccion: #Separamos la dirección en fragmentos
        nueva = str(conversor(fragmento)) #Pasamos el fragmento a decimal
        resultado.append(nueva) #Guardamos en la lista el resultado
    nueva = ".".join(resultado) #Unimos la dirección mediante puntos
    return nueva 

### Datagrama DNS

In [102]:
def protocolodns (contador, matriz): #Añadimos valores del protocolo DNS
    m.esdns = True
    m.dnstransid = "0x"+"".join(m.matriz[contador : contador+2])
    m.dnsflags = "0x"+"".join(m.matriz[contador+2 : contador+4])
    m.dnsquestions = "".join(m.matriz[contador+4 : contador+6])
    m.dnsquestions = conversor(m.dnsquestions)
    m.dnsanswer = "".join(m.matriz[contador+6 : contador+8])
    m.dnsanswer =  conversor(m.dnsanswer)
    m.dnsauthority = "".join(m.matriz[contador+8 : contador+10])
    m.dnsauthority = conversor(m.dnsauthority)
    m.dnsadditional = "".join(m.matriz[contador+10 : contador+12])
    m.dnsadditional = conversor(m.dnsadditional)
    m.dnsqueryname = "".join(m.matriz[contador+12 : contador+29])
    m.dnsquerytype = "".join(m.matriz[contador+29 : contador+31])
    m.dnsqueryclass = "".join(m.matriz[contador+31 : contador+33])
    contador += 33
    return contador
    

### Datagrama UDP

In [103]:
def protocoloudp(contador,matriz):  #Añadimos valores del protocolo UDP
    m.esudp = True
    m.udpsourceport = "".join(m.matriz[contador : contador+2])
    m.udpdestport = "".join(m.matriz[contador+2 : contador+4])
    m.udplength = "".join(m.matriz[contador+4 : contador+6])
    m.udplength = conversor(m.udplength)
    m.udpchecksum = "0x"+"".join(m.matriz[contador+6 : contador+8])
    contador += 8
    return contador

### Datagrama ICMP

In [104]:
def protocoloicmp(contador, matriz):  #Añadimos valores del protocolo ICMP
    m.esicmp=True
    m.icmptype = m.matriz[contador]
    m.icmptype = conversor(m.icmptype)
    m.icmpcode = m.matriz[contador+1]
    m.icmpcode = conversor(m.icmpcode)
    m.icmpchecksum = "0x"+"".join(m.matriz[contador+2:contador+4])
    m.icmpid = "".join(m.matriz[contador+4:contador+6])
    m.icmpid = conversor(m.icmpid)
    m.icmpseq = "".join(m.matriz[contador+6:contador+8])
    m.icmpseq = conversor(m.icmpseq)
    contador += 8
    return contador

### Flags TCP

In [105]:
def flagstcp(contador, matriz):  #Añadimos valores de los flags TCP
    m.tcpwinsizevalue = "".join(m.matriz[contador:contador+2]) 
    m.tcpwinsizevalue = conversor(m.tcpwinsizevalue)
    m.tcpchecksum = "0x"+"".join(m.matriz[contador+2:contador+4])
    m.tcpurgentpointer = "".join(m.matriz[contador+4:contador+6])
    m.tcpurgentpointer = conversor(m.tcpurgentpointer)
    contador += 6
    if m.tcpflag == '02': #Según el tipo de flag, tiene distintos atributos
        m.tcpmaxsegmentsize = "".join(m.matriz[contador:contador+4])
        m.tcpmaxsegmentsize = conversor(m.tcpmaxsegmentsize)
        m.tcpsackoption = m.matriz[contador+4:contador+6]
        m.tcptimestamps = m.matriz[contador+6:contador+16]
        m.tcpnop = m.matriz[contador+16]
        m.tcpwindowscale = m.matriz[contador+17:contador+20]
        contador += 19
            
    return contador

### Datagrama TCP

In [106]:
def protocolotcp(contador, matriz):  #Añadimos valores del protocolo TCP
    m.estcp=True
    m.tcpsourceport = "".join(m.matriz[contador:contador+2])
    m.tcpsourceport = conversor(m.tcpsourceport)
    m.tcpdestport = "".join(m.matriz[contador+2:contador+4])
    m.tcpdestport = conversor(m.tcpdestport)
    m.tcpsequencenumber = "".join(m.matriz[contador+4:contador+8])
    m.tcpsequencenumber = conversor(m.tcpsequencenumber)
    m.tcpacknowledgenumber = "".join(m.matriz[contador+8:contador+12])
    m.tcpacknowledgenumber = conversor(m.tcpacknowledgenumber)
    m.tcpheaderlenght = "".join(m.matriz[contador+12]) 
    m.tcpheaderlenght = conversor(m.tcpheaderlenght)
    m.tcpflag = m.matriz[contador+13]
    contador += 14
    
    contador = flagstcp(contador, matriz) #Tratamos los flags y los atributos posteriores
    return contador    

### Datagrama IP

In [107]:
def protocoloip (contador, matriz): #Añadimos valores del protocolo IP
    m.esip = True
    m.ipversion = m.matriz[contador][0]#Tipo de version ip
    m.ipsize = m.matriz[contador][1] #Tamaño del datagrama
    if m.ipversion == "4": #Tratamos los atributos correspondientes a la versión 4
        m.ipsize = int(m.ipsize)
        total = contador + (m.ipsize * 4)
        m.ipdifservfield = m.matriz[contador+1]
        m.iptotallen = "".join(m.matriz[contador+2:contador+4])
        m.iptotallen = conversor(m.iptotallen)
        m.ipid = "0x"+"".join(m.matriz[contador+4:contador+6])
        m.ipflags = "0x"+"".join(m.matriz[contador+6:contador+8])
        m.ipttl = "".join(m.matriz[contador+8])
        m.ipttl = conversor(m.ipttl)
        m.ipprotocolo = "".join(m.matriz[contador+9])
        m.ipprotocolo = conversor(m.ipprotocolo)
        m.ipheaderchecksum = "0x"+"".join(m.matriz[contador+10:contador+12])
        m.ipdest = m.matriz[contador+12:contador+16]
        m.ipdest = conversorip(m.ipdest)
        m.iporig = m.matriz[contador+16:contador+20]
        m.iporig = conversorip(m.iporig)
    elif m.ipversion == "6": #Tratamos los atributos correspondientes a la versión 6
        m.ipv6trafficclass = m.ipsize + "".join(m.matriz[contador+1:contador+4])
        m.iptotallen = "".join(m.matriz[contador+4:contador+6])
        m.iptotallen = conversor(m.iptotallen)
        m.ipv6nextheader = "".join(m.matriz[contador+6])
        m.ipv6nextheader = conversor(m.ipv6nextheader)
        m.ipv6hoplimit = "".join(m.matriz[contador+7])
        m.ipv6hoplimit = conversor(m.ipv6hoplimit)
        m.ipdest = ":".join(m.matriz[contador+8:contador+24])
        m.iporig = ":".join(m.matriz[contador+24:contador+40])
        total = contador +40
    return total

### Datagrama ARP

In [108]:
def protocoloarp(contador, matriz): #Añadimos valores del protocolo ARP
    m.esarp = True
    m.arphwtype = "".join(m.matriz[contador:contador+2])
    m.arphwtype = conversor(m.arphwtype)
    m.arpprotocoltype = "0x"+"".join(m.matriz[contador+2:contador+4])
    m.arphwsize = "".join(m.matriz[contador+4])
    m.arphwsize = conversor(m.arphwsize)
    m.arpprotocolsize = "".join(m.matriz[contador+5])
    m.arpprotocolsize = conversor(m.arpprotocolsize)
    m.arpopcoderequest = "".join(m.matriz[contador+6:contador+8])
    m.arpopcoderequest = conversor(m.arpopcoderequest)
    m.arpsourcemac = ":".join(m.matriz[contador+8:contador+14]) 
    m.arpsourceip = m.matriz[contador+14:contador+18]
    m.arpsourceip = conversorip(m.arpsourceip)
    m.arptargetmac = ":".join(m.matriz[contador+18:contador+24])
    m.arptargetip = m.matriz[contador+24:contador+28]
    m.arptargetip = conversorip(m.arptargetip)
    m.arppadding = "".join(m.matriz[contador+28:contador+46])
    contador +=46
    return contador

#### Separación de cabeceras y elementos del datagrama

In [109]:
n = 0 #Número de mensaje que estamos tratando
tipom = ""
for m in mensajesarchivo: #Vamos leyendo cada mensaje
    m.macdest = ":".join(m.matriz[0:6]) 
    m.macorig = ":".join(m.matriz[6:12])
    m.mactipo = "0x"+"".join(m.matriz[12:14])
    contador = 14
    #Dependiendo del tipo de mensaje que sea, se aplicará un protocolo u otro
    if m.mactipo == "0x0800": 
        contador = protocoloip(contador, m.matriz)
    elif m.mactipo == "0x86dd":
        contador = protocoloip(contador, m.matriz)
        contador = protocoloicmp(contador, m.matriz)
    elif m.mactipo == "0x0806":
        contador = protocoloarp(contador, m.matriz)
    if (m.ipprotocolo == 1):
        contador = protocoloicmp(contador, m.matriz)
    elif (m.ipprotocolo == 6):
        contador = protocolotcp(contador, m.matriz)
    elif (m.ipprotocolo == 17):
        contador = protocoloudp(contador, m.matriz)
        contador = protocolodns(contador, m.matriz)
    
    elementos(m)

ID MENSAJE:  1
Es IP:  False
Es ARP:  True
Es ICMP:  False
Es TCP:  False
Es UDP:  False
Es DNS:  False
MAC Destino:  ff:ff:ff:ff:ff:ff
MAC Origen:  08:00:27:dd:97:2e
Tipo MAC:  0x0806
Tipo hardware ARP:  1
Protocolo ARP:  0x0800
Tamaño Hardware ARP:  6
Ramaño protocolo ARP:  4
Code request ARP:  1
MAC origen ARP:  08:00:27:dd:97:2e
IP origen ARP:  192.168.1.1
MAC objetivo ARP:  00:00:00:00:00:00
IP objetivo ARP:  192.168.1.10
Padding ARP:  000000000000000000000000000000000000
------------------------------------------------- 

ID MENSAJE:  2
Es IP:  False
Es ARP:  True
Es ICMP:  False
Es TCP:  False
Es UDP:  False
Es DNS:  False
MAC Destino:  08:00:27:dd:97:2e
MAC Origen:  08:00:27:ac:fd:3d
Tipo MAC:  0x0806
Tipo hardware ARP:  1
Protocolo ARP:  0x0800
Tamaño Hardware ARP:  6
Ramaño protocolo ARP:  4
Code request ARP:  2
MAC origen ARP:  08:00:27:ac:fd:3d
IP origen ARP:  192.168.1.10
MAC objetivo ARP:  08:00:27:dd:97:2e
IP objetivo ARP:  192.168.1.1
Padding ARP:  
--------------------

#### Pseudocódigo

##### Definir reglas

In [110]:
import os
tcprules=[]
udprules=[]
iprules=[]
icmprules=[]
def reglas():
    ruta = "local.rules"
    f = open(ruta, "r") #Abrimos el documento generado
    for linea in f:
        if (linea[0] != "#" and linea != '\n'): #Comprobamos que la línea no sea ningún comentario o salto de línea
            #Dividimos las reglas según el protocolo que usen
            if(linea[6:8] == "ip"): 
                iprules.append(linea)
            elif(linea[6:10] == "icmp"):
                icmprules.append(linea)
            elif(linea[6:9]=="udp"):
                udprules.append(linea)
            elif(linea[6:9]=="tcp"):
                tcprules.append(linea)    
    f.close() 
    '''archivoreglas = os.listdir("/etc/snort/rules/") #Recorre todo el directorio
    for archivo in archivoreglas:
        ruta = "/etc/snort/rules/" + archivo #Nos metemos en un fichero del directorio
        f = open(ruta, "r") #Abrimos el documento generado
        for linea in f:
            if (linea[0] != "#" and linea != '\n'):
                if(linea[6:8] == "ip"):
                    iprules.append(linea)
                elif(linea[6:10] == "icmp"):
                    icmprules.append(linea)
                elif(linea[6:9]=="udp"):
                    udprules.append(linea)
                elif(linea[6:9]=="tcp"):
                    tcprules.append(linea)    
        f.close() '''
reglas()
print (iprules)
print (icmprules)
print (tcprules)
print (udprules)

[]
['alert icmp any any -> 192.168.1.1/24 any (msg:"PING Flooding"; ttl:100; dsize: > 200; sid:10000001; rev:1;) \n', 'alert icmp 192.168.1.3 any -> 192.168.1.0/24 any (msg:"Smurf attack detected"; itype:8; sid:5000002; rev:1;)']
['alert tcp any any -> any any (msg:"Land attack detected"; flags:S; sameip; sid:5000000; rev: 1;)\n', 'alert tcp any any -> 192.168.1.3 any (msg:"TCP SYN flood attack detected"; flags:S; threshold: type threshold, track by_dst, count 20 , seconds 60; sid: 5000001; rev:1;)\n']
['alert udp any any -> 192.168.1.3 any (msg:"UDP flood attack detected"; threshold: type threshold, track by_dst, count 10 , seconds 60 ; sid: 5000003; rev:1;)\n']


# Comparar el mesaje con las reglas y que se activen cuando haya match

### Análisis de reglas ip

In [21]:
def analisisip(m):
    for rule in iprules:
        sendm = False #Booleano que nos permite saber si se puede enviar la alerta
         
        msg = rule.split(";") #Separamos la regla para obtener la parte donde estáel mensaje
        msg = msg[0].split('"') #Separamos la parte del mensaje
        msg = msg[1] #Guardamos el mensaje
        regla = rule.split() #Separamos la regla para guardar direcciones y puertos
        
        dirorigen = regla[2]
        puertoorigen = regla[3]
        dirdest = regla[5]
        puertodest = regla[6]
        protocoloip = 0 #Protocolo IP utilizado
        operadorprotip = "" #Variable para coger el operador que aparece en el protocolo IP, si le hay
        listaprotocolos = [] #Si hay varios protocolos,se guardan en esta variable
        valido = False #Booleano para ver si aparecen varios protocolos
        ttlip = 0 #Variable para el tiempo de vida
        dsizeip= 0 #Variable para el tamaño de mensaje
        
        for i in regla:
            #Buscamos en la regla el protocolo ip
            if i.find('ip_proto') >= 0:
                protocoloip = i.split(":")
                protocoloip = protocoloip[1].split(";")
                protocoloip= protocoloip[0]
                if (protocoloip[0] == ">"): #Si aparece un signo de mayor, lo guardamos
                    operadorprotip = ">"
                    protocoloip = protocoloip[1:]
                elif (protocoloip[0] == "!"): #Si aparece un signo de desigualdad, lo guardamos
                    operadorprotip = "!"
                    listaprotocolos.append(protocoloip[1:])    
            #Buscamos en la regla el tiempo de vida
            if i.find('ttl') >= 0:
                ttlip = i.split(";")
                ttlip = ttlip[0].split(":")
                if(len(ttlip) == 2): 
                    ttlip = ttlip[1]
            #Buscamos en la regla el tamaño del mensaje        
            if i.find('dsize') >= 0:    
                dsizeip = i.split(";")
                dsizeip = dsizeip[0].split(":")
                dsizeip = dsizeip[1]
        
        #Comporbamos que las direcciones coincidan
        if ((dirorigen == "any" and dirdest =="any")
            or(dirorigen == "$EXTERNAL_NET"  and dirdest =="$HOME_NET")
               or (dirorigen == "any" and dirdest == m.ipdest)
                   or (dirorigen == m.iporig and dirdest =="any")):
            #Si existe el signo mayor en el protocolo, lo comparamos para ver si se cumple
            if(operadorprotip == ">"):
                if int(m.ipprotocolo) > int(protocoloip):
                    sendm = True
            elif(operadorprotip == "!"): #Si tiene el símbolo de desigual, lo tratamos
                for p in listaprotocolos: #Revisamos los protocolos para ver si hay coincidencia
                    if int(m.ipprotocolo) == int(p): #Si hay coincidencia, la regla no se cumple
                        valido = True
                if valido == False: #Si no ha habido coincidencia, se envía la alerta
                    sendm = True
            if(ttlip == m.ipttl or protocoloip == m.ipprotocolo or dsizeip == m.ipsize): #Si alguno de estos campos se cumple, se envía la alerta
                sendm = True
        
        elif (dirorigen == m.iporig and dirdest==m.ipdest): #Si las direcciones coinciden, se envía la alerta
            sendm = True
        
        if sendm == True: #Condicional para enviar la alerta
            print(msg)

In [84]:
def analisisicmp(m):
    for rule in icmprules:
        msg = rule.split(';') #Separamos las partes de la regla
        msg = msg[0].split('"') #Separamos el mensaje de la parte correspondiente
        msg = msg[1] #Guardamos el mensaje
        regla = rule.split() #Separamos la regla para guardar direcciones y puertos
        dirorigen = regla[2]
        puertoorigen = regla[3]
        dirdest = regla[5]
        puertodest = regla[6]
        #Si las direcciones tienen notaciones CIDR, las separamos de su dirección
        if dirorigen.find("/") >= 0:
            dirorigen = dirorigen.split("/")
            dirorigen = dirorigen[0]
        if dirdest.find("/") >= 0:
            dirdest = dirdest.split("/")
            dirdest = dirdest[0]
        
        ttl = "" #Tiempo de vida
        dsize= "" #Tamaño del mensaje
        dsizeoperator = "" #Operador del tamaño de mensaje
        itype = "" #Tipo de ICMP
        icode = "" #Codigo del ICMP
        icodeoperator = "" #Operador del código
        idicmp = "" #Id de ICMP
        seqicmp = "" #Secuencia de ICMP
        items = 0 #Contador de atributos encontrados en la regla
        for i in regla:
            #Buscamos tiempo de vida en la regla
            if i.find('ttl') >= 0:
                ttl = i.split(";")
                ttl = ttl[0].split(":")
                ttl = int(ttl[1])
                items += 1
            #Buscamos tamaño de paquete en la regla
            if i.find('dsize') >= 0: 
                dsize = i.split(";")
                dsize = dsize[0].split(":")
                dsize = dsize[1]
                if dsize == "": #Si no tiene valor, le asignamos el -1
                    dsize = -1
                elif dsize[0] == ">": #Si tiene operador de mayor, lo guardamos aparte
                    dsizeoperator = ">"
                    dsize = int(dsize[1:])
                else:
                    dsize = int(dsize)
                items += 1

            #Buscamos tipo de ICMP en la regla        
            if i.find('itype') >= 0: 
                itype = i.split(";")
                itype = itype[0].split(":")
                itype = int(itype[1])
                items += 1
            #Buscamos código de ICMP en la regla        
            if i.find('icode') >= 0: 
                icode = i.split(";")
                icode = icode[0].split(":")
                icode = icode[1]
                if icode[0] == ">": #Si tiene operador de mayor, lo guardamos aparte
                    icodeoperator = ">"
                    icode = int(icode[1:])
                else:
                    icode = int(icode)
                items += 1
            #Buscamos Id de ICMP en la regla    
            if i.find('icmp_id') >= 0: 
                idicmp = i.split(";")
                idicmp = idicmp[0].split(":")
                idicmp = int(idicmp[1])
                items += 1
            #Buscamos secuencia de ICMP en la regla    
            if i.find('icmp_seq') >= 0: 
                seqicmp = i.split(";")
                seqicmp = seqicmp[0].split(":")
                seqicmp = int(seqicmp[1])
                items += 1
                
        #Comporbamos que las direcciones coincidan
        if((dirorigen == "any" and dirdest == "any")
           or(dirorigen == "$EXTERNAL_NET" and dirdest =="$HOME_NET")
            or (dirorigen == "any" and m.ipdest == dirdest)
             or (m.iporig == dirorigen and dirdest == "any")):
            #Comprobamos que el ttl coincida
            if (ttl != "" and ttl == m.ipttl):
                items -= 1
            #Comprobamos que el tamaño coincida
            if (dsize != "" and m.ipsize == dsize):
                items -= 1
            else:
                if (dsize != "" and dsizeoperator == ">" and m.ipsize > dsize): #Si tiene comparador, hacemos la comparación
                    items -= 1
            #Comprobamos que el tipo coincida        
            if (itype != "" and itype == m.icmptype):
                items -= 1
            #Comprobamos que el código coincida
            if (icode != "" and m.icmpcode == icode):
                items -= 1
            else:
                if (icode != "" and icodeoperator == ">" and m.icmpcode > icode): #Si tiene comparador, hacemos la comparación
                    items -= 1    
            #Comprobamos que el ID coincida
            if (idicmp != "" and idicmp == m.icmpid):
                items -= 1
            #Comprobamos que la secuencia coincida
            if (seqicmp != "" and seqicmp == m.icmpseq):
                items -= 1
        #Si las direcciones origen y destino coinciden, se envía la alerta
        elif(m.iporig == dirorigen and m.ipdest == dirdest):
            print(msg)
        #Si el contador de atributos llega a 0, quiere decir que el mensaje cumple todos los atributos de la regla
        if items == 0:
            print(msg) #Enviamos la alerta

In [85]:
def analisistcp(m):
    for rule in tcprules:
        msg = rule.split(";") #Separamos las partes de la regla
        msg = msg[0].split('"') #Separamos el mensaje de la parte correspondiente
        msg = msg[1] #Guardamos el mensaje
        
        regla = rule.split() #Separamos la regla para guardar direcciones y puertos
        dirorigen = regla[2]
        puertoorigen = regla[3]
        dirdest = regla[5]
        puertodest = regla[6]
        
        neporig = False #Booleano para comprobar que hay un operador de desigualdad en el puerto de origen
        nepdest = False #Booleano para comprobar que hay un operador de desigualdad en el puerto de destino
        variosporig = False #Booleano para indicar que hay varios puertos de origen
        variospdest = False #Booleano para indicar que hay varios puertos de destino
        valido = False #Booleano para enviar el mensaje
        threshold = False #Booleano para indicar que está el atributo treshold
        trackbydest = False #Booleano para indicar que está el atributo track by_dest
        flags = "" #Variable para guardar el flag o flags
        count = 0 #Countador de mensajes
        for i in regla:
            print (i)
        
        #Comporbamos si el puerto origen tiene un comparador de desigualdad
        if(puertoorigen[0] == "!"):
            neporig = True
            puertoorigen = puertoorigen.split("!")
            puertoorigen = puertoorigen[1]
        #Comporbamos si el puerto destino tiene un comparador de desigualdad
        if(puertodest[0] == "!"):
            nepdest = True
            puertodest = puertodest.split("!")
            puertodest = puertodest[1]
        #Comprobamos si hay varios puertos origen
        if (puertoorigen.find(":") >= 0):
            variosporig = True
            puertoorigen = puertoorigen.split(":")
        #Comprobamos si hay varios puertos destino
        if (puertodest.find(":") >= 0):
            variospdest = True
            puertodest = puertodest.split(":")   

        #Comprobamos que coinciden las direcciones origen y destino
        if ((dirorigen == "any" and dirdest =="any")
            or(dirorigen == "$EXTERNAL_NET" and dirdest =="$HOME_NET")
               or(dirorigen == "$HOME_NET" and dirdest =="$EXTERNAL_NET")
                   or(dirorigen == "$HTTP_SERVERS" and dirdest =="$EXTERNAL_NET")
                    or(dirorigen == "$EXTERNAL_NET" and dirdest == "$SQL_SERVERS")
                       or(dirorigen=="$EXTERNAL_NET" and dirdest == "$SMTP_SERVERS")
                           or (dirorigen == "any" and m.ipdest == dirdest)
                             or (m.iporig == dirorigen and dirdest == "any")):
            #Comprobamos que coinciden los puertos origen y destino
            if((puertoorigen == "any" and puertodest == "any")
               or (puertoorigen == "any" and puertodest == "$HTTP_PORTS")
               or (puertoorigen == "$HTTP_PORTS" and puertodest == "any")):
                valido=True
            else:
                #Puertos destino
                if(variospdest): #Comprobamos si hay varios puertos de destino para establecer un intervalo
                    if (puertodest[0] == ""): #Si el primer puerto está vacío, comprobamos si hay comprador de desigualdad
                        if(nepdest and m.tcpdestport > int(puertodest[1])): #Si hay desigualdad, comprobamos que el puerto sea mayor al limite
                            valido = True
                        elif ((nepdest == False) and m.tcpdestport <= int(puertodest[1])):#Si no hay desigualdad, comprobamos que el puerto sea menor o igual al limite
                            valido = True
                    elif (puertodest[1] == ""):
                        if(nepdest and m.tcpdestport < int(puertodest[1])):#Si hay desigualdad, comprobamos que el puerto sea menor al limite
                            valido = True
                        elif ((nepdest == False) and m.tcpdestport >= int(puertodest[1])): #Si no hay desigualdad, comprobamos que el puerto sea mayor o igual al limite
                            valido = True  
                    else: #Si incluye los dos puertos, comprobamos que esté en el intervalo
                        if (m.tcpdestport >= int(puertodest[0]) and m.tcpdestport <= int(puertodest[1])):
                            valido = True 
                else:
                    #Si hay un comparador de desigualdad en el puerto de destino lo tratamos
                    if(nepdest):
                        if (m.tcpdestport != int(puertodest)):
                            valido = True
                    else: #Si no hay comparador, lo tratamos
                        if (m.tcpdestport == int(puertodest)):
                            valido = True
                            
                #Puertos origen                
                if(variosporig): #Comprobamos si hay varios puertos de origen para establecer un intervalo
                    if (puertoorigen[0] == ""): #Si el primer puerto está vacío
                        if(neporig and m.tcpsourceport > int(puertoorigen[1])): #Si hay desigualdad, comprobamos que el puerto sea mayor al limite
                            valido = True
                        elif ((neporig == False) and m.tcpsourceport <= int(puertoorigen[1])): #Si no hay desigualdad, comprobamos que el puerto sea menor o igual al limite
                            valido = True
                    elif (puertoorigen[1] == ""):#Si el segundo puerto está vacío
                        if(neporig and m.tcpsourceport < int(puertoorigen[1])): #Si hay desigualdad, comprobamos que el puerto sea menor al limite
                            valido = True
                        elif ((neporig == False) and (m.tcpsourceport >= int(puertoorigen[1]))):#Si no hay desigualdad, comprobamos que el puerto sea mayor o igual al limite
                            valido = True  
                    else: #Si incluye los dos puertos, comprobamos que esté en el intervalo
                        if (m.tcpsourceport >= int(puertoorigen[0]) and m.tcpsourceport <= int(puertoorigen[1])):
                            valido = True 
                else:
                    #Si hay un comparador de desigualdad en el puerto de origen lo tratamos
                    if(neporig):
                        if (m.tcpsourcetport != int(puertoorigen)):
                            valido = True
                    else: #Si no hay comparador, lo tratamos
                        if (m.tcpsourceport == int(puertoorigen)):
                            valido = True
        #Si coinciden las direcciones de origen y destino se envía la alerta
        elif (dirorigen == m.iporig and dirdest==m.ipdest):
                valido = True
        if valido == True:
            print(msg)    

In [112]:
def analisisudp(m):        
     for rule in udprules:    
        msg = rule.split(';') #Separamos todos los fragmentos de ; que haya
        regla = msg #Guardamos el resultado para más adelante
        msg = msg[0].split('"') #Separamos las comillas para extraer el mensaje de alerta
        msg = msg[1] #Guardamos en la variable el mensaje de alerta

        regla = regla[1].split(",") #Separamos las comas de los apartados a examinar
        
        valido = False #Booleano que nos indicará si el paquete es válido para enviar una alerta
        threshold= False #Campo treshold 
        trackbydest = False #Campo track by_dest
        count = "" #Número limite de mensajes para considerarlo amenaza
        seg = "" #Intervalo de segundos para considerarlo una amenaza
        
        for i in regla: #Miramos dentro de la alerta para ver si existen los elementos necesarios
            if (i.find("threshold: type threshold")>= 0):
                threshold = True
            if (i.find("track by_dst")>= 0):
                trackbydest = True
            if (i.find("count") >= 0):
                count = i.split()
                count = count[1]
                count = int(count)
            if (i.find("seconds") >= 0):
                seg = i.split()
                seg = seg[1]
                seg = int(seg)
        
        regla = rule.split() #Separamos la alerta de nuevo para extraer las direcciones y puertos   
        dirorigen = regla[2]
        puertoorigen = regla[3]
        dirdest = regla[5]
        puertodest = regla[6]
        
        #En este condicional las direcciones deben ser adecuadas para emitir una alerta
        if((dirorigen == "any" and dirdest == "any")
                or (dirorigen == "any" and m.ipdest == dirdest)
                 or (m.iporig == dirorigen and dirdest == "any")):
            #Si los campos están activados, se procede a examinar los puertos para emitir la alerta
            if (treshold and trackbydest and (contador > count)):                
                if (puertoorigen == m.udpsourceport and puertodes == m.udpdestport):
                    valido = True
                elif (puertoorigen == m.udpsourceport and puertodes == "any"):
                    valido = True
                elif (puertoorigen == "any" and puertodest == m.udpdestport):
                    valido = True
                elif (puertoorigen == "any" and puertodest == "any"):
                    valido = True
            # Si no existen esos campos, procedemos a ver si los puertos coinciden 
            else:
                if (puertoorigen == m.udpsourceport and puertodes == m.udpdestport):
                    valido = True
                elif (puertoorigen == m.udpsourceport and puertodes == "any"):
                    valido = True
                elif (puertoorigen == "any" and puertodest == m.udpdestport):
                    valido = True
                elif (puertoorigen == "any" and puertodest == "any"):
                    valido = True
        #Si el booleano valido está activo, se emite la alerta
        if (valido):
            print(msg)
        

In [113]:
def analisis(m):
    if (m.esarp):
        print("Es un mensaje ARP")
    if (m.esip):
        print("Es un mensaje IP")
        analisisip(m)
    if(m.esicmp):
        print("Es un mensaje ICMP")
        analisisicmp(m)
    if (m.estcp):
        print("Es un mensaje TCP")
        analisistcp(m)
    if (m.esudp):
        print("Es un mensaje UDP")
        analisisudp(m)
    if (m.esdns):
        print("Es un mensaje DNS")    
        

In [114]:
for m in mensajesarchivo:
    analisis(m)
    print('\n')


Es un mensaje ARP


Es un mensaje ARP


Es un mensaje IP
Es un mensaje ICMP


Es un mensaje IP
Es un mensaje ICMP


Es un mensaje IP
Es un mensaje ICMP


Es un mensaje IP
Es un mensaje ICMP


Es un mensaje IP
Es un mensaje ICMP


Es un mensaje IP
Es un mensaje ICMP


Es un mensaje IP
Es un mensaje ICMP


Es un mensaje IP
Es un mensaje ICMP


Es un mensaje ARP


Es un mensaje ARP


Es un mensaje IP
Es un mensaje UDP
Es un mensaje DNS


Es un mensaje IP
Es un mensaje UDP
Es un mensaje DNS




## Por hacer
+ Añadir todos los tipos de mensajes existentes
+ Comparar reglas con mensajes y que envíen alertas (En forma de mensaje):
    - Mirar campos de la práctica de snort
    - Intentar hacer mach con esos campos (entre mensaje y alerta predefinida)


## Hecho
- Cambiar la manera de leer los archivos para que los lean enteros
- Crear una clase y una constructora para los mensajes
+ Añadir reglas estándar

In [83]:
for rule in udprules:
    print (rule)

alert udp any any -> 192.168.1.3 any (msg:"UDP flood attack detected"; threshold: type threshold, track by_dst, count 10 , seconds 60 ; sid: 5000003; rev:1;)

